In [1]:
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import datetime
import os
import time
import re
import sys
import pandas as pd
from lxml import html

# declare global variables
driver = None
counter = None
limit = None
cached_list = None
df = None

In [2]:
def sanitize(s):
    return s.lower().replace(" ", "").replace("-", "").replace(",", "").replace(":", "").strip()

In [3]:
def prep_driver():
    """
    Sets up a webdriver instance of chrome and navigates to the appropriate webpage.
    Returns the driver instance (so it doesn't get garbage collected on function exit)
    """
    global driver
    global counter
    global limit
    global cached_list
    global df

    # Configure chrome webdriver
    if os.name == 'nt':
        chromedriver = "./chromedriver.exe"
    else:
        chromedriver = "./chromedriver"
    os.environ["webdriver.chrome.driver"] = chromedriver
    driver = webdriver.Chrome(chromedriver)

    # Navigate to SIMS website
    driver.get('https://sims.me.exova.com/SIMS/SIMS-MVC/DashBoard/Index')

    # Enter username and password
    selectElem=driver.find_element_by_id("UserName")
    selectElem.clear()
    selectElem.send_keys("vignesh.v")
    selectElem=driver.find_element_by_name("pwd")
    selectElem.clear()
    selectElem.send_keys("Adjan2017#")

    # Press 'Login'
    selectElem=driver.find_element_by_xpath('//*[@id="LoginMain"]/div/div[3]/form/button')
    selectElem.click()

    # Navigate to test method section
    driver.get('https://sims.me.exova.com/SIMS/SIMS-MVC/Product/Index#ProductSearchGrid')

    #Expand 'Test Search'
    selectElem=driver.find_element_by_xpath('//a[@href="#ProductSearchGrid"]')
    selectElem.click()
    return driver

In [4]:
def search(method):
    """Searches a test method and returns the results in a dataframe"""
    global driver
    global counter
    global limit
    global cached_list
    global df

    method_filename = method.replace(":","..") # Windows is unable to parse filenames with a colon ":", replace those with two dots ".." instead

    # if cached results for the method search already exists, return it instead of doing the serach again
    if method_filename in cached_list:
        print("")
        print("{} already exists in cached_list".format(method_filename))
        # if a csv file is empty, return an empty dataframe instead (to avoid exception when parsing an empty csv file)
        if os.path.getsize("./data/{}.csv".format(method_filename)) > 0:
            return pd.read_csv("./data/{}.csv".format(method_filename), header=None, index_col=None, escapechar="|")
        else:
            return pd.DataFrame()

    # Clear the search field
    selectElem=driver.find_element_by_xpath('//*[@id="ProductSearch"]/div[1]/div/table/thead/tr[2]/th[3]/span/span/span/input')
    selectElem.clear()
    selectElem.send_keys(Keys.ENTER)
    time.sleep(1)

    # Search the method
    selectElem=driver.find_element_by_xpath('//*[@id="ProductSearch"]/div[1]/div/table/thead/tr[2]/th[3]/span/span/span/input')
    selectElem.clear()
    selectElem.send_keys(method.strip())
    selectElem.send_keys(Keys.ENTER)
    selectElem=driver.find_element_by_xpath('//*[@id="ProductSearch"]/div[2]').click()
    time.sleep(2)

    # Obtain the table results from the page
    soup = BeautifulSoup(driver.page_source, features="lxml")
    t1 = soup.find("div", {"id":"ProductSearch"}).find("div", {"class":"k-grid-content"}).find("table").find("tbody")
    # i = 1

    # Send table's html to file for debugging purposes
    with open("output1.html", "w") as file:
        file.write(str(t1.prettify()))

    # Transcribe the html table into a dataframe
    tbl_list = []
    tmp_list = []
    for row in t1.findAll("tr"):
        i = 0
        tmp_list = []
        for cell in row.findAll("td"):
            if (i >= 3):
                break
            else:
                i += 1
                #print(i)
                tmp_list.append(cell.text.strip())
        #print(tmp_list)
        tbl_list.append(tmp_list)
    df = pd.DataFrame(tbl_list)

    # cache the results of this search into a csv file and update the cached_list
    df.to_csv("./data/{}.csv".format(method.replace(":","..")), header=None, index=None, escapechar="|")
    cached_list.append(method)
    print("")
    print("{} added to cached_list".format(method.replace(":","..")))

    return df

In [5]:
def first_pass(test_id, test_item, test_method):
    """
    searches for exact matches
    """
    global driver
    global counter
    global limit
    global cached_list
    global df

    if limit>0 and counter>=limit:
        counter+=1
        return "blank"

    # if the 'test_id' column of df isn't already filled with a test id then do a search
    if pd.notnull(test_id) and re.compile("[A-Z0-9]{6}").match(test_id):
        counter+=1
        return test_id
    res=search(test_method.strip())

    # track progress
    print("[{}/{}] hits: {} ({},{})"
          .format(
              counter+1,
              limit if limit>0 else df.shape[0],
              res.shape[0],
              test_method.strip(),
              test_item.strip()))
#     breakpoint()
    with pd.option_context('display.max_rows', None, 'display.max_columns', None, 'display.max_colwidth', -1):
        print(res)

    #res[0]: res_id
    #res[1]: res_item
    #res[2]: res_method
    if res.shape[0] == 1:
        if sanitize(res.iloc[0,1]) == sanitize(test_item):
            counter+=1
            print("Perfect Match: {} | {}".format(res.iloc[0,1], test_item))
            return res.iloc[0,0]
        elif (sanitize(res.iloc[0,1]) in sanitize(test_item) or sanitize(test_item) in sanitize(res.iloc[0,1])):
            counter+=1
            print("{} | {}".format(res.iloc[0,1], test_item))
            return res.iloc[0,0]
        else:
            counter+=1
            print("{} | {}".format(res.iloc[0,1], test_item))
            return "{}:{}".format(res.iloc[0,0], res.iloc[0,1])

    elif res.shape[0] > 1:
        res_exact=res[res[2] == test_method.strip()]
        print("exact hits: {}".format(res_exact.shape[0]))
        with pd.option_context('display.max_rows', None, 'display.max_columns', None, 'display.max_colwidth', -1):
            print(res_exact)
        if res_exact.shape[0] == 1:
            if sanitize(res_exact.iloc[0,1]) == sanitize(test_item):
                counter+=1
                print("Perfect Match: {} | {}".format(res_exact.iloc[0,1], test_item))
                return res_exact.iloc[0,0]
            elif (sanitize(res_exact.iloc[0,1]) in sanitize(test_item) or sanitize(test_item) in sanitize(res_exact.iloc[0,1])):
                counter+=1
                print("{} | {}".format(res.iloc[0,1], test_item))
                return res_exact.iloc[0,0]
            else:
                print("{} | {}".format(res_exact.iloc[0,1], test_item))
                counter+=1
                return "{}:{}".format(res_exact.iloc[0,0], res_exact.iloc[0,1])
        else:
            counter+=1
            print("{} exact hits".format(res_exact.shape[0]))
            return "{} exact hits".format(res_exact.shape[0])

    else:
        counter+=1
        print("{} hits on initial search".format(res.shape[0]))
        return "{} hits".format(res.shape[0])
    counter+=1
    print("you should never reach this part of first_pass()")
    return "you should never reach this part of first_pass()"

In [6]:
def main():
    global driver
    global counter
    global limit
    global cached_list
    global df

    df=pd.read_excel("./r2.xlsx", header=None, usecols=[0,1])
    if df.shape[1] == 2:
        df.insert(0,"","")
        df.columns = ["test_id","test_item","test_method"]
    elif df.shape[1] == 3:
        """do nothing"""
    else:
        sys.exit("first_pass() received a dataframe with {} columns (only 2 or 3 is allowed)".format(df.shape[1]))
    df

    # Initialize the chrome webdriver instance
    try:
        driver.current_url
    except:
        driver=prep_driver()
    # driver = ""

    # Initialize the counter and limit. Leave limit as a negative number if you want to loop through the entire 
    counter=0
    limit=-1

    # Note down the available cached search results in a list
    cached_list = [f for f in os.listdir("./data")]
    cached_list = [os.path.splitext(f)[0] for f in cached_list]
    cached_list = []

    df['test_id'] = df.apply(
            lambda row: first_pass(row['test_id'],row['test_item'],row['test_method']),
            axis=1
        )
    counter=0 # reset the counter back to 0

    # export the resultant dataframe to excel
    # df.to_excel("./df1.xlsx", header=None, index=None)
    df

In [7]:
if __name__ == '__main__':
    main()


BS EN 932-3 added to cached_list
[1/2] hits: 2 (BS EN 932-3,Petrographic Examination of Aggregates for Concrete)
        0                         1            2
0  ASO158  Petrographic Analysis     BS EN 932-3
1  ASO255  Petrographic Examination  BS EN 932-3
exact hits: 2
        0                         1            2
0  ASO158  Petrographic Analysis     BS EN 932-3
1  ASO255  Petrographic Examination  BS EN 932-3
2 exact hits

BS 7943 added to cached_list
[2/1] hits: 1 (BS 7943,Petrographic Examination of Aggregates for Concrete)
        0                                                    1        2
0  T00938  Petrographic Examination of Aggregates for Concrete  BS 7943
Perfect Match: Petrographic Examination of Aggregates for Concrete | Petrographic Examination of Aggregates for Concrete

BS 812-104 added to cached_list
[3/1] hits: 1 (BS 812-104,Petrographic Examination of Aggregates for Concrete)
        0                         1           2
0  ASO210  Petrographic Examinatio


SS 73 Cl 13 Method B added to cached_list
[12/1] hits: 1 (SS 73 Cl 13 Method B,Fines Content (Clay, Silt and Dust) - Decantation Method)
        0                                                         1  \
0  T00992  Fines Content (Clay, Silt and Dust) - Decantation Method   

                      2  
0  SS 73 Cl 13 Method B  
Perfect Match: Fines Content (Clay, Silt and Dust) - Decantation Method | Fines Content (Clay, Silt and Dust) - Decantation Method

SS 73 Cl 14 Method C added to cached_list
[13/1] hits: 1 (SS 73 Cl 14 Method C,Fines Content (Clay, Silt and Dust) - Field Settling Test)
        0                                                          1  \
0  T00993  Fines Content (Clay, Silt and Dust) - Field Settling Test   

                      2  
0  SS 73 Cl 14 Method C  
Perfect Match: Fines Content (Clay, Silt and Dust) - Field Settling Test | Fines Content (Clay, Silt and Dust) - Field Settling Test

BS 812-103.1 already exists in cached_list
[14/1] hits: 3 (BS 812-


BS EN 1097-6 added to cached_list
[23/4] hits: 4 (BS EN 1097-6,Particle Density & Water Absorption)
        0                                    1                   2
0  ASO051  Density & Water Absorption           BS EN 1097-6, Cl 9
1  ASO082  Density & Water Absorption           BS EN 1097-6, Cl 8
2  ASO258  Specific gravity & Water Absorption  BS EN 1097-6      
3  ASO177  Thermal and weathering properties    BS EN 1097-6      
exact hits: 2
        0                                    1             2
2  ASO258  Specific gravity & Water Absorption  BS EN 1097-6
3  ASO177  Thermal and weathering properties    BS EN 1097-6
2 exact hits

BS 812-2 already exists in cached_list
[24/4] hits: 9 (BS 812-2,Particle Density & Water Absorption)
        0                                           1  \
0  ASO119  Bulk Density (Compacted and Loose) & Voids   
1  ASO239  Particle Density                             
2  ASO156  Particle density  & water absorption         
3  ASO157  Particle dens


ASTM C40 added to cached_list
[31/5] hits: 5 (ASTM C40,Organic Impurites)
        0                                                               1  \
0  CSR061  Organic Impurities                                               
1  CHM043  Organic impurities in sand                                       
2  ASO191  Organic Impurities in Sand                                       
3  T00005  Setting Time                                                     
4  CON011  Time of Setting of Concrete Mixtures by Penetration Resistance   

                 2  
0  ASTM C40         
1  ASTM C40: 2004   
2  ASTM C40         
3  ASTM C403        
4  ASTM C403/C403M  
exact hits: 2
        0                           1         2
0  CSR061  Organic Impurities          ASTM C40
2  ASO191  Organic Impurities in Sand  ASTM C40
2 exact hits

SS 73 already exists in cached_list
[32/5] hits: 22 (SS 73,Organic Impurites)
         0                                                          1  \
0   T00193  1


BS 812-105.2 added to cached_list
[40/1] hits: 1 (BS 812-105.2,Elongation Index)
        0                 1             2
0  ASO132  Elongation Index  BS 812-105.2
Perfect Match: Elongation Index | Elongation Index

SS 73 already exists in cached_list
[41/1] hits: 22 (SS 73,Aggregate Impact Value)
         0                                                          1  \
0   T00193  10% Fines Value (TFV)                                       
1   T00131  Acid Soluble Sulphate Content                               
2   T00194  Aggregate Crushing Value (ACV)                              
3   T00998  Aggregate Impact Value                                      
4   T01009  Angularity Number                                           
5   T00994  Bulk Density (Compacted and Loose) & Voids                  
6   T00932  Drying Shrinkage                                            
7   T00996  Elongation Index                                            
8   T00992  Fines Content (Clay, Silt and 


BS 812-111 added to cached_list
[46/1] hits: 1 (BS 812-111,10% Fines Value)
        0                        1           2
0  ASO174  Ten percent fines value  BS 812-111
Ten percent fines value | 10% Fines Value

SS 73 already exists in cached_list
[47/1] hits: 22 (SS 73,Los Angeles Abrasion)
         0                                                          1  \
0   T00193  10% Fines Value (TFV)                                       
1   T00131  Acid Soluble Sulphate Content                               
2   T00194  Aggregate Crushing Value (ACV)                              
3   T00998  Aggregate Impact Value                                      
4   T01009  Angularity Number                                           
5   T00994  Bulk Density (Compacted and Loose) & Voids                  
6   T00932  Drying Shrinkage                                            
7   T00996  Elongation Index                                            
8   T00992  Fines Content (Clay, Silt and Dust) 


SS 73 Part 20 added to cached_list
[56/1] hits: 1 (SS 73 Part 20,Drying Shrinkage)
        0                 1              2
0  T00932  Drying Shrinkage  SS 73 Part 20
Perfect Match: Drying Shrinkage | Drying Shrinkage

BS EN 1367-4 added to cached_list
[57/0] hits: 0 (BS EN 1367-4,Drying Shrinkage)
Empty DataFrame
Columns: []
Index: []
0 hits on initial search

BS 812-120 added to cached_list
[58/1] hits: 1 (BS 812-120,Drying Shrinkage)
        0                 1           2
0  ASO129  Drying shrinkage  BS 812-120
Perfect Match: Drying shrinkage | Drying Shrinkage

BS EN 933-9 added to cached_list
[59/1] hits: 1 (BS EN 933-9,Assessment of Fines - Methylene Blue Test)
        0                                          1            2
0  CHE018  Assessment of Fines - Methylene Blue Test  BS EN 933-9
Perfect Match: Assessment of Fines - Methylene Blue Test | Assessment of Fines - Methylene Blue Test

BS EN 933-4 added to cached_list
[60/1] hits: 1 (BS EN 933-4,Shape Index)
        0   


BS EN 1097-8 added to cached_list
[74/3] hits: 3 (BS EN 1097-8,Polished Stone Value)
        0                               1                     2
0  T01014  Aggregate Abrasion Value        BS EN 1097-8 Annex A
1  T01038  Aggregate Abrasion Value (AAV)  BS EN 1097-8 Annex 8
2  ASO244  Polished Stone Value (PSV)      BS EN 1097-8        
exact hits: 1
        0                           1             2
2  ASO244  Polished Stone Value (PSV)  BS EN 1097-8
Aggregate Abrasion Value | Polished Stone Value

BS 812-114 added to cached_list
[75/1] hits: 1 (BS 812-114,Polished Stone Value)
        0                       1           2
0  ASO113  Polished - Stone Value  BS 812-114
Perfect Match: Polished - Stone Value | Polished Stone Value

ASTM C123 added to cached_list
[76/2] hits: 2 (ASTM C123,Lightweight Particles in Aggregate)
        0                              1          2
0  CHE016  Lightweight Particle Analysis  ASTM C123
1  ASO143  Lightweight pieces             ASTM C123
exact h


BS EN 1744-1 Cl 15.1 added to cached_list
[95/1] hits: 1 (BS EN 1744-1 Cl 15.1,Potential Presence of Humus)
        0                            1                     2
0  T00939  Potential Presence of Humus  BS EN 1744-1 Cl 15.1
Perfect Match: Potential Presence of Humus | Potential Presence of Humus

BS EN 1744-1 Cl 15.2 added to cached_list
[96/1] hits: 1 (BS EN 1744-1 Cl 15.2,Fulvo Acid Content)
        0                   1                     2
0  T01020  Fulvo Acid Content  BS EN 1744-1 Cl 15.2
Perfect Match: Fulvo Acid Content | Fulvo Acid Content

BS EN 1744-1 Cl 17 added to cached_list
[97/0] hits: 0 (BS EN 1744-1 Cl 17,Loss on Ignition)
Empty DataFrame
Columns: []
Index: []
0 hits on initial search

ASTM C637 Cl 9.1.3.1 added to cached_list
[98/1] hits: 1 (ASTM C637 Cl 9.1.3.1,Fixed Water Content by Ignition Test)
        0                                     1                     2
0  T01021  Fixed Water Content by Ignition Test  ASTM C637 Cl 9.1.3.1
Perfect Match: Fixed W

In [9]:
df_z = df.copy()

In [14]:
df=pd.read_excel("./r2.xlsx", header=None, usecols=[0,1])
df

,0,1
0,Petrographic Examination of Aggregates for Con...,BS EN 932-3
1,Petrographic Examination of Aggregates for Con...,BS 7943
2,Petrographic Examination of Aggregates for Con...,BS 812-104 ...
3,Petrographic Examination of Aggregates for Con...,ASTM C295
4,Potential Alkali Reactivity (Mortar Bar Method),ASTM C1260
5,Particle Size Distribution (Sieve Analysis),SS 73
6,Particle Size Distribution (Sieve Analysis),ASTM C136
7,Particle Size Distribution (Sieve Analysis),BS 812-103.1
8,Particle Size Distribution (Sieve Analysis),BS EN 933-1
9,Particle Size Distribution (Sieve Analysis),ASTM D546 (ASTM D242)


In [16]:
if df.shape[1] == 2:
    df.insert(0,"","")
    df.columns = ["test_id","test_item","test_method"]
elif df.shape[1] == 3:
    """do nothing"""
else:
    sys.exit("first_pass() received a dataframe with {} columns (only 2 or 3 is allowed)".format(df.shape[1]))
df

,test_id,test_item,test_method
0,,Petrographic Examination of Aggregates for Con...,BS EN 932-3
1,,Petrographic Examination of Aggregates for Con...,BS 7943
2,,Petrographic Examination of Aggregates for Con...,BS 812-104 ...
3,,Petrographic Examination of Aggregates for Con...,ASTM C295
4,,Potential Alkali Reactivity (Mortar Bar Method),ASTM C1260
5,,Particle Size Distribution (Sieve Analysis),SS 73
6,,Particle Size Distribution (Sieve Analysis),ASTM C136
7,,Particle Size Distribution (Sieve Analysis),BS 812-103.1
8,,Particle Size Distribution (Sieve Analysis),BS EN 933-1
9,,Particle Size Distribution (Sieve Analysis),ASTM D546 (ASTM D242)
